In [1]:
! python -m pip install numpy matplotlib pandas scipy seaborn scikit-learn tensorflow keras_tuner google-genai pypdf

  Using cached numpy-2.4.1-cp313-cp313-win_amd64.whl.metadata (6.6 kB)
  Using cached matplotlib-3.10.8-cp313-cp313-win_amd64.whl.metadata (52 kB)
  Using cached pandas-3.0.0-cp313-cp313-win_amd64.whl.metadata (19 kB)
  Using cached scipy-1.17.0-cp313-cp313-win_amd64.whl.metadata (60 kB)
  Using cached seaborn-0.13.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached scikit_learn-1.8.0-cp313-cp313-win_amd64.whl.metadata (11 kB)
  Using cached tensorflow-2.20.0-cp313-cp313-win_amd64.whl.metadata (4.6 kB)
  Using cached keras_tuner-1.4.8-py3-none-any.whl.metadata (5.6 kB)
  Using cached google_genai-1.60.0-py3-none-any.whl.metadata (53 kB)
  Using cached pypdf-6.6.0-py3-none-any.whl.metadata (7.1 kB)
  Using cached contourpy-1.3.3-cp313-cp313-win_amd64.whl.metadata (5.5 kB)
  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
  Using cached fonttools-4.61.1-cp313-cp313-win_amd64.whl.metadata (116 kB)
  Using cached kiwisolver-1.4.9-cp313-cp313-win_amd64.whl.metadata (6.4 kB)
  


[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
! python -m pip install streamlit

  Using cached altair-6.0.0-py3-none-any.whl.metadata (11 kB)
  Using cached blinker-1.9.0-py3-none-any.whl.metadata (1.6 kB)
  Using cached click-8.3.1-py3-none-any.whl.metadata (2.6 kB)
  Using cached pandas-2.3.3-cp313-cp313-win_amd64.whl.metadata (19 kB)
  Using cached toml-0.10.2-py2.py3-none-any.whl.metadata (7.1 kB)
  Using cached watchdog-6.0.0-py3-none-win_amd64.whl.metadata (44 kB)
  Using cached gitpython-3.1.46-py3-none-any.whl.metadata (13 kB)
  Using cached pydeck-0.9.1-py2.py3-none-any.whl.metadata (4.1 kB)
  Using cached narwhals-2.15.0-py3-none-any.whl.metadata (13 kB)
  Using cached gitdb-4.0.12-py3-none-any.whl.metadata (1.2 kB)
  Using cached pytz-2025.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached smmap-5.0.2-py3-none-any.whl.metadata (4.3 kB)
   ---------------------------------------- 0.0/9.1 MB ? eta -:--:--
   -------- ------------------------------- 1.8/9.1 MB 10.8 MB/s eta 0:00:01
   ------------------ --------------------- 4.2/9.1 MB 10.9 MB/s eta 0:


[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
from google import genai
from google.genai import types
from pypdf import PdfReader
import numpy as np

In [ ]:
api_key = '---'
client = genai.Client(api_key=api_key)

SpaceBot - Ask me anything about space.

I den här delen av kunskapskontrollen ska jag skapa en chattbot med RAG. All fakta och information som chattbotten använder kommer från en PDF-fil från NASA om vårt solsystem. Chattbotten ska kunna svara på allt från enkla frågor till mer detaljerade frågor om vårat solsystem. 

I koden läses PDF-filen in, som chattbotten använder som informationskälla.

In [3]:
reader = PdfReader("pdf_file/solarsystem.pdf")

text = ""

for page in reader.pages:
    text += page.extract_text()

Chunks

Delar texten i mindre chunks så att modellen kan hantera den enklare.

In [4]:
chunks = []
n = 800
overlap = 150

for i in range(0, len(text), n - overlap):
    chunks.append(text[i:i + n])

print(f"Amount of chunks: {len(chunks)}.")

Amount of chunks: 224.


För att få chattbotten att prestera bättre har jag rensat PDF:en genom att ta bort radbrytningar, brutna ord, länkar och upprepande ord. Chunksen ska innehålla text som är relevant.

In [5]:
chunks_clean = []

for chunk in chunks:

    chunk = chunk.replace("\n", " ").replace("- ", "").strip()
    
    if(
       len(chunk) > 400
       and chunk.count(".") >= 2
       and "www.nasa.gov" not in chunk
       and "OUR SOLAR SYSTEM" not in chunk.upper()

    ):
        chunks_clean.append(chunk)
    
print(f"Amount of chunks after cleaning: {len(chunks_clean)}")

Amount of chunks after cleaning: 184


Printar ut några chunks.

In [6]:
for i, chunk in enumerate(chunks_clean[:3]):
    print(f"\n--- Clean Chunk {i+1} ---\n")
    print(chunk)


--- Clean Chunk 1 ---

n — and these are the  names astronomers use today. Planetary features are named by  the International Astronomical Union, founded in 1919. For more  information about the names of planets, moons, and features,  consult the Gazetteer of Planetary Nomenclature website at  planetarynames.wr.usgs.gov. Ancient observers believed that the Sun and all the other celestial bodies revolved around Earth. Astronomers gradually  realized that the Earth-centered model did not account for the  motions of the planets. In the early 17th century, Galileo Galilei’s discoveries using the recently invented telescope strongly  supported the concept of a “solar system” in which all the planets, including Earth, revolve around a central star — the Sun.  Planetary moons, the rings of Saturn, and more planet

--- Clean Chunk 2 ---

e four  planets closest to the Sun — Mercury, Venus, Earth, and Mars —  are called the terrestrial planets because they have solid, rocky  surfaces. Two of t

Printar ut en specifik chunk.

In [7]:
print(chunks_clean[50])

r terrestrial planets. Since the ancient time of volcanism, the arid, lifeless Moon has  remained nearly unchanged. With too sparse an atmosphere  to impede impacts, a steady rain of asteroids, meteoroids, and  comets strikes the surface. Over billions of years, the surface  has been ground up into fragments ranging from huge boulders  to powder. Nearly the entire Moon is covered by a rubble pile of  charcoal-gray, powdery dust and rocky debris called the lunar  regolith. Beneath is a region of fractured bedrock referred to as  the megaregolith.  The Moon was first visited by the U.S.S.R.’s Luna 1 and 2 in  1959, and a number of U.S. and U.S.S.R. robotic spacecraft  followed. The U.S. sent three classes of robotic missions to prepare the way for human exploration: the Rangers (1961–1965)


Embedding

I detta steg skapas embeddnings för varje chunk, så att AI:n kan jämföra dem och hitta semantisk relevant text för frågor.

In [8]:
def create_embeddings(text_list, model="text-embedding-004", task_type="SEMANTIC_SIMILARITY"):
    embeddings = []
    for i in range(0, len(text_list), 100):  
        resp = client.models.embed_content(
            model=model,
            contents=text_list[i:i+100],
            config=types.EmbedContentConfig(task_type=task_type)
        )
        embeddings += resp.embeddings  
    return embeddings

In [9]:
embeddings = create_embeddings(chunks_clean)
print(len(embeddings))

184


Semantisk Search

Funktionen nedan jämför frågan med alla chunks och returnerar sedan de mest relevanta chunks för att svara på frågan. 

In [10]:
def cosine_similarity(vec1, vec2):
    return (np.dot(vec1, vec2) / (np.linalg.norm(vec1)*np.linalg.norm(vec2)))

In [11]:
def semantic_search(query, chunks_clean, embeddings, k=5):
    query_embedding = create_embeddings([query])[0].values  
    
    similarity_scores = []
    
    for i, chunk_embedding in enumerate(embeddings):
        similarity_score = cosine_similarity(query_embedding, chunk_embedding.values)
        similarity_scores.append((i, similarity_score))
        
    similarity_scores.sort(key=lambda x: x[1], reverse=True)
    top_indices = [index for index, _ in similarity_scores[:k]]
    
    return [chunks_clean[index] for index in top_indices]

En fråga skickas och en semantisk sökning hittar den mest relevanta text i chunksen.

In [12]:
question = "Tell me information about Jupyter?"
answer = semantic_search(question, chunks_clean, embeddings=embeddings, k=1)
print(answer)

['TS Namesake   King of the Roman gods  Mean Distance from the Sun 778.41 million km    (483.68 million mi)  Orbit Period 11.8565 Earth years    (4,330.6 Earth days)  Orbit Eccentricity (Circular Orbit = 0) 0.04839  Orbit Inclination to Ecliptic 1.305 deg  Inclination of Equator to Orbit 3.12 deg  Rotation Period 9.92 hr  Equatorial Radius 71,492 km (44,423 mi)  Mass 317.82 of Earth’ s  Density 1.33 g/cm3  Gravity 20.87 m/sec2 (68.48 ft/sec2)  Atmosphere Primary Components hydr ogen, helium  Effective Temperature at 1 bar  –108 deg C (–163 deg F)  Known Moons* 50  Rings 1 (thr ee major components) *Plus 17 awaiting official confirmation, total 67, as of July 2013. SIGNIFICANT DATES 1610 — Galileo Galilei makes the first detailed observations of  Jupiter.  1973 — Pioneer 10 becomes the first']


In [13]:
question = "Tell me about Earth?"
answer = semantic_search(question, chunks_clean, embeddings=embeddings, k=1)
print(answer)

['f space.  Earth is made up of complex, interactive systems that create a  constantly changing world that we are striving to understand.  From the vantage point of space, we are able to observe our  planet globally, using sensitive instruments to understand the  delicate balance among its oceans, air, land, and life. NASA satellite observations help study and predict weather, drought, pollution, climate change, and many other phenomena that affect  the environment, economy, and society.  Earth is the third planet from the Sun and the fifth largest in the  solar system. Earth’s diameter is just a few hundred kilometers  larger than that of Venus. The four seasons are a result of Earth’s  axis of rotation being tilted 23.45 degrees with respect to the  plane of Earth’s orbit around the Su']


Generera Svar

Skapar en prompt som  kan svara på allt som finns i PDF:en. Om information saknas i PDF:en ska den svara att den inte vet och inte gissa.

In [14]:
system_prompt = """
I will ask you a question, and I want you to answer based on the given context and no other information. 
If there is not enough information, respond exactly like this:
"I don't know." 
Do not try to guess or answer.
Keep your answers simple and break them down to nice paragrahs.
Do not write phrases like "Based on the context" or anything similiar """

Här skapar jag en användarprompt som kombinerar frågan med relevant information från texten.

In [15]:
def generate_user_prompt(query):
    context = "\n".join(semantic_search(query, chunks_clean, embeddings))

    user_prompt = f"Question is {query}. Here is the context: {context}"

    return user_prompt

Här skickas en systemprompt och en användares fråga till modellen som sedan returnerar ett svar.

In [16]:
def generate_response(system_prompt, user_message, model="gemini-2.0-flash"):
    response = client.models.generate_content(
        model=model,
        config=genai.types.GenerateContentConfig(
            system_instruction=system_prompt),
            contents=generate_user_prompt(user_message)
    )

    return response

Ställer frågor som det finns information om.

In [17]:
print(generate_response(system_prompt,"How many moons does earth have?").text)

Earth has 1 moon.



In [18]:
print(generate_response(system_prompt,"How many planets exist in our solarsystem?").text)

There are eight planets in our solar system.



In [19]:
print(generate_response(system_prompt,"Which minerals exists in mars?").text)

Igneous rocks are crystallized from magma.
Iron-nickel meteorites exist on Mars.



Ställer frågor om information som inte finns i kontexten.

In [20]:
print(generate_response(system_prompt,"Give me recipe with pasta?").text)

I don't know.


In [21]:
print(generate_response(system_prompt,"What is Deep Learning?").text)

I don't know.



Evaluering

In [22]:
validation_data = [
    {
        "question": "How large is Mercury's metallic core compared to the planet's radius?",
        "ideal_answer": """Mercury's metallic core has a radius of about 2,000 kilometers (1,240 miles), 
        which is about 80 percent of the planet’s total radius."""
    }
]

print(validation_data[0]["question"])
print()
print(validation_data[0]["ideal_answer"])

How large is Mercury's metallic core compared to the planet's radius?

Mercury's metallic core has a radius of about 2,000 kilometers (1,240 miles), 
        which is about 80 percent of the planet’s total radius.


Det här är en system-prompt som fungerar som ett utvärderingssystem. Den ger oss poäng (1, 0.5 eller 0) baserat på hur bra AI-modellens svar matchar det önskade svaret.

In [23]:
evaluation_system_prompt = """
You are a smart evaluation system tasked with evaluating the anwsers given by an AI assistant.
If the answer is fully correct, respond exactly with: Score : 1
If the answer is incorrect, respond exactly with: Score : 0
If the answer is partially correct, respond exactly with: Score : 0.5
Explain briefly why you gave that score.
"""

Här skickas frågan till modellen, genererar ett svar, och skickar sedan både AI-svaret och det önskade svaret till modellen för utvärdering, och skriver sedan ut utvärderingsresultatet.

In [24]:
query = validation_data[0]["question"]

response = generate_response(system_prompt, query)

evaluation_prompt = f"""Question: {query}
AI-answer: {response.text}
Ideal answers: {validation_data[0]['ideal_answer']}"""
 
evaluation_response = generate_response(evaluation_system_prompt, evaluation_prompt)
print(evaluation_response.text)

Score : 1
The AI assistant provided a completely correct answer, accurately stating the size of Mercury's metallic core in kilometers and as a percentage of the planet's radius.



In [25]:
validation_data_1 = [
    {
        "question": "Tell me something about machine learning?",
        "ideal_answer": "Machine learning is a type of AI where computers learn patterns from data to make predictions or decisions."
    }
]

print(validation_data_1[0]["question"])
print()
print(validation_data_1[0]["ideal_answer"])

Tell me something about machine learning?

Machine learning is a type of AI where computers learn patterns from data to make predictions or decisions.


In [26]:
query = validation_data_1[0]["question"]

response = generate_response(system_prompt, query)

evaluation_prompt = f"""Question: {query}
AI-answer: {response.text}
Ideal answers: {validation_data_1[0]['ideal_answer']}"""
 
evaluation_response = generate_response(evaluation_system_prompt, evaluation_prompt)
print(evaluation_response.text)

Score : 0
The AI's answer "I don't know" indicates a lack of information or inability to answer the question.



En enkel applikation.

In [28]:
print("SpaceBot - Ask me anything about space")

while True:
    question = input("")
    if question.lower() == "q":
        break

    print("\nQuestion:", question)
    answer = generate_response(system_prompt, question)
    print("\nAI-answer:", answer.text, "\n")


SpaceBot - Ask me anything about space

Question: How many rings does saturn have?

AI-answer: Saturn has 7 main rings (C, B, A, D, F, G, E).
 


Question: Which planet is biggest?

AI-answer: Jupiter is the biggest planet.
 



Här exporterar jag chattbottens chunks och embeddings för att kunna använda det i streamlit.

In [29]:
import pickle

rag_data = {
    "chunks_clean": chunks_clean,
    "embeddings": [e.values for e in embeddings]
}

with open("rag_bot.pkl", "wb") as f:
    pickle.dump(rag_data, f)